In [1]:
#Loading Packages
from matplotlib import ticker
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
from textblob import TextBlob

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joypy
import re
import networkx as nx

In [2]:
#Loading the dataset
df = pd.read_excel('Cleaned_DS.xlsx')
df.head()

,localID,extractionDate,Name,Country,globalRating,positionOnRanking,sitesOnRanking,totalReviews,userName,userLocation,userContributions,tripType,reviewWritten,reviewVisited,reviewRating,reviewFullText
0,MAG001,2021-08-20 08:24:40.077,Basilica of the Sagrada Familia,Spain,4.5,1,1186,163828,Maurice M@MauriceM171,United States,29,Couples,2021-08-18,2021-08-01,5,"Absolutely breathtaking use of light, an amazi..."
1,MAG001,2021-08-20 08:24:40.097,Basilica of the Sagrada Familia,Spain,4.5,1,1186,163828,nickchamb@nickchamb,Ireland,7,Couples,2021-08-17,2021-03-01,5,A really interesting and beautiful place to vi...
2,MAG001,2021-08-20 08:24:40.117,Basilica of the Sagrada Familia,Spain,4.5,1,1186,163828,Arthur Villon@arthur4588,Spain,268,Friends,2021-08-16,2021-08-01,5,Beautiful place and must visit place in barcel...
3,MAG001,2021-08-20 08:24:40.147,Basilica of the Sagrada Familia,Spain,4.5,1,1186,163828,James U@696jamesu,Thailand,7,Family,2021-08-16,2021-07-01,5,A MUST when visiting Barcelkona. It is consta...
4,MAG001,2021-08-20 08:24:40.159,Basilica of the Sagrada Familia,Spain,4.5,1,1186,163828,Kamal K Sirra@Sirra63,Spain,516,Family,2021-08-15,2021-07-01,4,I visited about 20 years ago and there has bee...


In [3]:
#Summary statistics for all variables
df.describe(include='all').T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
localID,78409,99,MAG001,7355,NaN,NaN,NaN,NaN,NaN,NaN,NaN
extractionDate,78409,NaN,NaN,NaN,2021-08-20 22:18:04.136967680,2021-08-20 08:24:40.077000,2021-08-20 12:05:27.017999872,2021-08-20 15:46:32.840999936,2021-08-21 12:35:16.383000064,2021-08-21 16:27:11.893000,NaN
Name,78409,98,Basilica of the Sagrada Familia,7355,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Country,78409,24,Spain,19219,NaN,NaN,NaN,NaN,NaN,NaN,NaN
globalRating,78409.0,NaN,NaN,NaN,4.484976,4.0,4.5,4.5,4.5,5.0,0.178734
positionOnRanking,78409.0,NaN,NaN,NaN,3.88722,1.0,1.0,2.0,6.0,55.0,4.85389
sitesOnRanking,78409.0,NaN,NaN,NaN,734.407721,5.0,154.0,484.0,1186.0,3837.0,786.328371
totalReviews,78409.0,NaN,NaN,NaN,40974.763025,5179.0,14152.0,24454.0,51324.0,163828.0,43543.153689
userName,78409,60992,KodoDrummer@KodoDrummer,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN
userLocation,78409,118,United Kingdom,29921,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Apriori Algorithm

* In order to have more refined and meaningfull patterns, we decided to filter the dataset to give us only results for people were the frequency of reviews is higher then 7.

* To do that, we first had to create a new column in our dataset to calculate the user frequency.

In [4]:
df['userFreq'] = df.groupby('userName')['userName'].transform('count')
df['ratioFreqContribution'] = df['userFreq'] / df['userContributions']
dfRatio = df[df['userContributions'] != 0] #We needed to add this so that we don't divide results by 0

In [5]:
#Adding extra columns to perform "cosine" function
df["averageRating"] = df.groupby('userName')['reviewRating'].transform('mean')
df['diffAvgRating'] = df['reviewRating'] - df['averageRating']

In [6]:
#Filtering user Frequency by 7 to perfom the Apriori algorithm
df_apriori = df[df['userFreq'] > 7]

In [7]:
#Creating a pivot table
pt = pd.pivot_table(df_apriori[['userName','Name']],
                    index='userName',                                  # Each row will be a document (invoice)
                    columns='Name',                                    # Each product will be a column
                    aggfunc=lambda x: 1 if len(x)>0 else 0).fillna(0)  # The value should be 1 if the product is present in the document, otherwise 0
pt

Name,Abbaye du Mont-Saint-Michel,Acropolis,Acueduct of Segovia,Alcázar of Segovia,Ancient City of Ephesus,Arena di Verona,Basilica Papale e Sacro Convento...,Basilica of the Sagrada Familia,Bath Abbey,Blenheim Palace,...,Torre de Belém,Tower of Hercules (Torre de Hercules),Tower of London,Valle dei Templi,Vatican City,Vieux Lyon,Villa d'Este,Vilnius Old Town,Westminster Abbey,Wieliczka Salt Mine
userName,,,,,,,,,,,,,,,,,,,,,
Adventure beyond imagination@steelriver2014,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ana G@anagP6520XJ,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
Atefehdanesh@Atefehdanesh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B1714D@B1714D,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BackinBracknell@BackinBracknell,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
raguanum@raguanum,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
staledale@staledale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ucartech@ucartech,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
frequent_itemsets = apriori(pt.astype('bool'), min_support=0.05, use_colnames=True)

##  Analyzing which attractions are Visited Together

In [9]:
# Generate the association rules - by support
rulesSupport = association_rules(frequent_itemsets, metric="support", min_threshold=0.20) 
rulesSupport.sort_values(by='confidence', ascending=False, inplace=True) #sorted by confidence in order to know which are the location visited toghter whit higher confidence
rulesSupport.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1,(Casa Batllo),(Basilica of the Sagrada Familia),0.226415,0.405660,0.216981,0.958333,2.362403,0.125133,14.264151,0.745493
6,(Real Alcazar de Sevilla),(Catedral de Sevilla),0.226415,0.273585,0.207547,0.916667,3.350575,0.145603,8.716981,0.906874
2,(Casa Milà - La Pedrera),(Basilica of the Sagrada Familia),0.235849,0.405660,0.207547,0.880000,2.169302,0.111873,4.952830,0.705387
5,(Parc Guell),(Basilica of the Sagrada Familia),0.283019,0.405660,0.235849,0.833333,2.054264,0.121040,3.566038,0.715789
7,(Catedral de Sevilla),(Real Alcazar de Sevilla),0.273585,0.226415,0.207547,0.758621,3.350575,0.145603,3.204852,0.965762
4,(Basilica of the Sagrada Familia),(Parc Guell),0.405660,0.283019,0.235849,0.581395,2.054264,0.121040,1.712788,0.863492
0,(Basilica of the Sagrada Familia),(Casa Batllo),0.405660,0.226415,0.216981,0.534884,2.362403,0.125133,1.663208,0.970324
3,(Basilica of the Sagrada Familia),(Casa Milà - La Pedrera),0.405660,0.235849,0.207547,0.511628,2.169302,0.111873,1.564690,0.906926


## Analyzing which combination of countries has the highest support

In [10]:
# Create a pivot table
pt_country = pd.pivot_table(df_apriori[['userName','Country']],
                    index='userName',                                  # Each row will be a document (invoice)
                    columns='Country',                                 # Each product will be a column
                    aggfunc=lambda x: 1 if len(x)>0 else 0).fillna(0)  # The value should be 1 if the product is present in the document, otherwise 0
pt_country.head()

Country,Austria,Belgium,Bulgaria,Croatia,Czech Republic,England,Estonia,Finland,France,Germany,...,Latvia,Lithuania,Montenegro,Norway,Poland,Portugal,Russia,Scotland,Spain,Turkey
userName,,,,,,,,,,,,,,,,,,,,,
Adventure beyond imagination@steelriver2014,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
Ana G@anagP6520XJ,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
Atefehdanesh@Atefehdanesh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
B1714D@B1714D,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
BackinBracknell@BackinBracknell,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [11]:
frequent_country = apriori(pt_country.astype('bool'), min_support=0.01, use_colnames=True)

#### This code bellow shows us that it's very frequent that when a user visits Spain, they also visit Portugal.
We will use this as an oportunity to create a Marketing strategy.

In [12]:
rulesSupportISO = association_rules(frequent_country, metric="support", min_threshold=0.06)
rulesSupportISO.sort_values(by='support', ascending=False, inplace=True)
rulesSupportISO.head(200)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
81,(Portugal),(Spain),0.349057,0.603774,0.283019,0.810811,1.342905,0.072268,2.094340,0.392271
80,(Spain),(Portugal),0.603774,0.349057,0.283019,0.468750,1.342905,0.072268,1.225305,0.644444
30,(England),(Italy),0.358491,0.330189,0.179245,0.500000,1.514286,0.060876,1.339623,0.529412
31,(Italy),(England),0.330189,0.358491,0.179245,0.542857,1.514286,0.060876,1.403302,0.507042
38,(England),(Scotland),0.358491,0.235849,0.179245,0.500000,2.120000,0.094696,1.528302,0.823529
...,...,...,...,...,...,...,...,...,...,...
21,(Poland),(Croatia),0.198113,0.207547,0.066038,0.333333,1.606061,0.024920,1.188679,0.470588
20,(Croatia),(Poland),0.207547,0.198113,0.066038,0.318182,1.606061,0.024920,1.176101,0.476190
17,(England),(Croatia),0.358491,0.207547,0.066038,0.184211,0.887560,-0.008366,0.971394,-0.164912
66,(Russia),(Italy),0.226415,0.330189,0.066038,0.291667,0.883333,-0.008722,0.945616,-0.145833


#### Creating a dataset containing only users that have visited Spain but not Portugal

In [13]:
reviewed_spain = df['Country'] == 'Spain' #Users From Spain

reviewed_portugal = df['Country'] == 'Portugal' #Users From Portugal

#Creating a list of unique usernames who have reviewed Spain and Portugal
users_reviewed_spain = df.loc[reviewed_spain, 'userName'].unique()
users_reviewed_portugal = df.loc[reviewed_portugal, 'userName'].unique()

#Converting these lists into sets for efficient set operations
users_reviewed_spain_set = set(users_reviewed_spain)
users_reviewed_portugal_set = set(users_reviewed_portugal)

#Finding the difference between the two sets to get users who have reviewed Spain but not Portugal
users_reviewed_spain_not_portugal = users_reviewed_spain_set - users_reviewed_portugal_set

#Converting the set back into a list
visted_ES_no_PT = list(users_reviewed_spain_not_portugal)


In [14]:
# Filter the original DataFrame to include only reviews from users who reviewed Spain but not Portugal
df_visted_ES_no_PT = df[df['userName'].isin(visted_ES_no_PT)]
df_visted_ES_no_PT

,localID,extractionDate,Name,Country,globalRating,positionOnRanking,sitesOnRanking,totalReviews,userName,userLocation,userContributions,tripType,reviewWritten,reviewVisited,reviewRating,reviewFullText,userFreq,ratioFreqContribution,averageRating,diffAvgRating
0,MAG001,2021-08-20 08:24:40.077,Basilica of the Sagrada Familia,Spain,4.5,1,1186,163828,Maurice M@MauriceM171,United States,29,Couples,2021-08-18,2021-08-01,5,"Absolutely breathtaking use of light, an amazi...",1,0.034483,5.00,0.00
1,MAG001,2021-08-20 08:24:40.097,Basilica of the Sagrada Familia,Spain,4.5,1,1186,163828,nickchamb@nickchamb,Ireland,7,Couples,2021-08-17,2021-03-01,5,A really interesting and beautiful place to vi...,2,0.285714,5.00,0.00
2,MAG001,2021-08-20 08:24:40.117,Basilica of the Sagrada Familia,Spain,4.5,1,1186,163828,Arthur Villon@arthur4588,Spain,268,Friends,2021-08-16,2021-08-01,5,Beautiful place and must visit place in barcel...,1,0.003731,5.00,0.00
3,MAG001,2021-08-20 08:24:40.147,Basilica of the Sagrada Familia,Spain,4.5,1,1186,163828,James U@696jamesu,Thailand,7,Family,2021-08-16,2021-07-01,5,A MUST when visiting Barcelkona. It is consta...,1,0.142857,5.00,0.00
4,MAG001,2021-08-20 08:24:40.159,Basilica of the Sagrada Familia,Spain,4.5,1,1186,163828,Kamal K Sirra@Sirra63,Spain,516,Family,2021-08-15,2021-07-01,4,I visited about 20 years ago and there has bee...,2,0.003876,4.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78383,MAG100,2021-08-21 16:26:54.150,Centro Storico,Italy,4.5,11,677,5179,Raoulr89@Raoulr89,Netherlands,727,Family,2019-10-28,2019-10-01,5,I think this is one of the most beautiful old ...,4,0.005502,4.75,0.25
78384,MAG100,2021-08-21 16:26:59.297,Centro Storico,Italy,4.5,11,677,5179,davidandmel@davidandmel,United Kingdom,1258,Friends,2019-10-13,2019-10-01,4,No trip to this magical city would be complete...,3,0.002385,4.00,0.00
78385,MAG100,2021-08-21 16:26:59.393,Centro Storico,Italy,4.5,11,677,5179,G_EvansWales@G_EvansWales,United Kingdom,848,Family,2019-10-09,2019-09-01,5,What can be written about this place that has ...,4,0.004717,5.00,0.00
78393,MAG100,2021-08-21 16:27:00.144,Centro Storico,Italy,4.5,11,677,5179,ARodriguezP@ARodriguezP,United States,1582,Family,2019-08-07,2019-08-01,5,Filled with many historical and religious site...,4,0.002528,4.75,0.25


The objective of the code bellow is to be used to understand which attractions in Spain and Portugal are perceived similarly by tourists, potentially indicating similar visitor experiences, themes, or satisfaction levels. 

This information is valuable since we know that the highest support level in our dataset is between Spain and Portugal. The objective is that by checking the similiraty between atractions that the Turists already visited we can recomend them the atractions they will probably like to visit in Portugal. 

In [15]:
from sklearn.metrics import pairwise_distances

# Filter the DataFrame for Spain and Portugal
df_spain = df[df['Country'] == 'Spain']
df_portugal = df[df['Country'] == 'Portugal']

# Create a pivot table for Spain
pt_spain = pd.pivot_table(df_spain,
                          index='userName',
                          columns='Name',
                          values='diffAvgRating',
                          aggfunc='mean').fillna(0)

# Create a pivot table for Portugal
pt_portugal = pd.pivot_table(df_portugal,
                             index='userName',
                             columns='Name',
                             values='diffAvgRating',
                             aggfunc='mean').fillna(0)

# Align the two pivot tables to have the same set of users
common_users = pt_spain.index.intersection(pt_portugal.index)
pt_spain_aligned = pt_spain.loc[common_users]
pt_portugal_aligned = pt_portugal.loc[common_users]

# Calculate cosine similarity between LocalIDs of Spain and Portugal
# Since we want to compare the LocalIDs, we work with the transpose of the pivot tables
similarity_matrix = pd.DataFrame(
    pairwise_distances(pt_spain_aligned.T, pt_portugal_aligned.T, metric='cosine'),
    columns = pt_portugal_aligned.columns,
    index = pt_spain_aligned.columns
)

# Transform dissimilarity to similarity
similarity_matrix = similarity_matrix.apply(lambda x: 1-x, axis=1)

similarity_matrix


Name,Bom Jesus do Monte,Cais da Ribeira,Mosteiro dos Jeronimos,Park and National Palace of Pena,Ponte de Dom Luís I,Quinta da Regaleira,Torre de Belém
Name,,,,,,,
Acueduct of Segovia,0.000000,0.000588,-0.037381,-0.091195,0.069518,0.000553,-0.104727
Alcázar of Segovia,0.000000,0.000919,0.049298,-0.109457,0.000667,0.000865,-0.111608
Basilica of the Sagrada Familia,0.004852,-0.078167,-0.195916,-0.107205,0.105551,-0.137453,-0.269448
Casa Batllo,0.000000,0.000000,-0.083799,0.011596,-0.101992,-0.166393,-0.056108
Casa Milà - La Pedrera,0.002158,-0.090389,-0.115764,-0.071959,-0.091654,0.023201,-0.009303
Catedral De Burgos,0.000000,0.000000,0.000000,0.055461,0.097683,0.000000,0.000000
Catedral Primada,0.000000,0.006438,-0.035413,-0.032429,0.004673,-0.068389,-0.042458
Catedral de Sevilla,0.019175,0.049833,0.024473,-0.048842,-0.033366,-0.014220,-0.197443
Cathedral De Santiago de Compostela,-0.464468,-0.086895,-0.016016,-0.099630,-0.016975,0.088970,-0.113421


Now that we have the similarity values we will order descending order to check the highest values

In [16]:
#Creating an empty list to store the similarities
similarity_list = []

#Iterate over each cell in the similarity matrix
for spain_id in similarity_matrix.index:
    for portugal_id in similarity_matrix.columns:
        similarity_score = similarity_matrix.loc[spain_id, portugal_id]
        similarity_list.append((spain_id, portugal_id, similarity_score))

#Converting the list to a DataFrame
similarity_df = pd.DataFrame(similarity_list, columns=['Spain_LocalID', 'Portugal_LocalID', 'Similarity'])

#Sortin g the DataFrame by similarity score in descending order
sorted_similarity_df = similarity_df.sort_values(by='Similarity', ascending=False)

#Checking the similarity
top_similar_pairs = sorted_similarity_df.head(20)





In [17]:
# This keeps the first occurrence of each Spanish attraction
# Drop duplicates from the top_5_similar_pairs DataFrame
top_similar_pairs_no_duplicates = top_similar_pairs.drop_duplicates(subset=['Spain_LocalID'])

top_similar_pairs_no_duplicates


,Spain_LocalID,Portugal_LocalID,Similarity
69,La Lonja de la Seda,Torre de Belém,0.133673
82,Palace of Catalan Music,Quinta da Regaleira,0.106347
18,Basilica of the Sagrada Familia,Ponte de Dom Luís I,0.105551
39,Catedral De Burgos,Ponte de Dom Luís I,0.097683
61,Cathedral De Santiago de Compostela,Quinta da Regaleira,0.088970
71,Mezquita Cathedral de Cordoba,Cais da Ribeira,0.081096
4,Acueduct of Segovia,Ponte de Dom Luís I,0.069518
96,Real Alcazar de Sevilla,Quinta da Regaleira,0.058904
50,Catedral de Sevilla,Cais da Ribeira,0.049833
9,Alcázar of Segovia,Mosteiro dos Jeronimos,0.049298


In [18]:
#Selecting only the desired columns
df_visted_ES_no_PT_filtered = df_visted_ES_no_PT[['localID', 'Name', 'Country', 'userName','reviewRating']]

df_visted_ES_no_PT_filtered



,localID,Name,Country,userName,reviewRating
0,MAG001,Basilica of the Sagrada Familia,Spain,Maurice M@MauriceM171,5
1,MAG001,Basilica of the Sagrada Familia,Spain,nickchamb@nickchamb,5
2,MAG001,Basilica of the Sagrada Familia,Spain,Arthur Villon@arthur4588,5
3,MAG001,Basilica of the Sagrada Familia,Spain,James U@696jamesu,5
4,MAG001,Basilica of the Sagrada Familia,Spain,Kamal K Sirra@Sirra63,4
...,...,...,...,...,...
78383,MAG100,Centro Storico,Italy,Raoulr89@Raoulr89,5
78384,MAG100,Centro Storico,Italy,davidandmel@davidandmel,4
78385,MAG100,Centro Storico,Italy,G_EvansWales@G_EvansWales,5
78393,MAG100,Centro Storico,Italy,ARodriguezP@ARodriguezP,5


In [19]:
#Reordering the columns in the desired order
df_visted_ES_no_PT_reordered = df_visted_ES_no_PT_filtered[['userName', 'Name', 'localID', 'Country','reviewRating']]

df_visted_ES_no_PT_reordered


,userName,Name,localID,Country,reviewRating
0,Maurice M@MauriceM171,Basilica of the Sagrada Familia,MAG001,Spain,5
1,nickchamb@nickchamb,Basilica of the Sagrada Familia,MAG001,Spain,5
2,Arthur Villon@arthur4588,Basilica of the Sagrada Familia,MAG001,Spain,5
3,James U@696jamesu,Basilica of the Sagrada Familia,MAG001,Spain,5
4,Kamal K Sirra@Sirra63,Basilica of the Sagrada Familia,MAG001,Spain,4
...,...,...,...,...,...
78383,Raoulr89@Raoulr89,Centro Storico,MAG100,Italy,5
78384,davidandmel@davidandmel,Centro Storico,MAG100,Italy,4
78385,G_EvansWales@G_EvansWales,Centro Storico,MAG100,Italy,5
78393,ARodriguezP@ARodriguezP,Centro Storico,MAG100,Italy,5


In [20]:
#Creating a conditional column to add the recomendations for each user

def recommend_attraction(name):
    if name == "Basilica of the Sagrada Familia":
        return "Ponte de Dom Luís I"
    elif name == "La Lonja de la Seda":
        return "Torre de Belém"
    elif name == "Palace of Catalan Music":
        return "Quinta da Regaleira"
    elif name == "Catedral De Burgos":
        return "Ponte de Dom Luís I"
    elif name == "Cathedral De Santiago de Compostela":
        return "Quinta da Regaleira"
    elif name == "Mezquita Cathedral de Cordoba":
        return "Cais da Ribeira"
    elif name == "Acueduct of Segovia":
        return "Ponte de Dom Luís I"
    elif name == "Real Alcazar de Sevilla":
        return "Quinta da Regaleira"
    elif name == "Catedral de Sevilla":
        return "Cais da Ribeira"
    elif name == "Alcázar of Segovia":
        return "Mosteiro dos Jeronimos"
    elif name == "The Walls of Avila":
        return "Park and National Palace of Pena"
    elif name == "Tower of Hercules (Torre de Hercules)":
        return "Cais da Ribeira"
    elif name == "Casa Milà - La Pedrera":
        return "Quinta da Regaleira"
    else:
        return None  # or some default value

# Apply the function to the DataFrame using .loc
df_visted_ES_no_PT_reordered.loc[:, 'Recommended_Attraction'] = df_visted_ES_no_PT_reordered['Name'].apply(recommend_attraction)

df_visted_ES_no_PT_reordered

C:\Users\Madalena\AppData\Local\Temp\ipykernel_1888\3610787817.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_visted_ES_no_PT_reordered.loc[:, 'Recommended_Attraction'] = df_visted_ES_no_PT_reordered['Name'].apply(recommend_attraction)


,userName,Name,localID,Country,reviewRating,Recommended_Attraction
0,Maurice M@MauriceM171,Basilica of the Sagrada Familia,MAG001,Spain,5,Ponte de Dom Luís I
1,nickchamb@nickchamb,Basilica of the Sagrada Familia,MAG001,Spain,5,Ponte de Dom Luís I
2,Arthur Villon@arthur4588,Basilica of the Sagrada Familia,MAG001,Spain,5,Ponte de Dom Luís I
3,James U@696jamesu,Basilica of the Sagrada Familia,MAG001,Spain,5,Ponte de Dom Luís I
4,Kamal K Sirra@Sirra63,Basilica of the Sagrada Familia,MAG001,Spain,4,Ponte de Dom Luís I
...,...,...,...,...,...,...
78383,Raoulr89@Raoulr89,Centro Storico,MAG100,Italy,5,None
78384,davidandmel@davidandmel,Centro Storico,MAG100,Italy,4,None
78385,G_EvansWales@G_EvansWales,Centro Storico,MAG100,Italy,5,None
78393,ARodriguezP@ARodriguezP,Centro Storico,MAG100,Italy,5,None


In [21]:
#Reordering the columns in the desired order
Marketing_Team_Suggestion = df_visted_ES_no_PT_reordered[['userName', 'Name','Recommended_Attraction','localID', 'Country','reviewRating']]

Marketing_Team_Suggestion

,userName,Name,Recommended_Attraction,localID,Country,reviewRating
0,Maurice M@MauriceM171,Basilica of the Sagrada Familia,Ponte de Dom Luís I,MAG001,Spain,5
1,nickchamb@nickchamb,Basilica of the Sagrada Familia,Ponte de Dom Luís I,MAG001,Spain,5
2,Arthur Villon@arthur4588,Basilica of the Sagrada Familia,Ponte de Dom Luís I,MAG001,Spain,5
3,James U@696jamesu,Basilica of the Sagrada Familia,Ponte de Dom Luís I,MAG001,Spain,5
4,Kamal K Sirra@Sirra63,Basilica of the Sagrada Familia,Ponte de Dom Luís I,MAG001,Spain,4
...,...,...,...,...,...,...
78383,Raoulr89@Raoulr89,Centro Storico,None,MAG100,Italy,5
78384,davidandmel@davidandmel,Centro Storico,None,MAG100,Italy,4
78385,G_EvansWales@G_EvansWales,Centro Storico,None,MAG100,Italy,5
78393,ARodriguezP@ARodriguezP,Centro Storico,None,MAG100,Italy,5


In [22]:
#Preparing dataset with relevant information for extraction
Mkt_sug = Marketing_Team_Suggestion[
    (Marketing_Team_Suggestion['Country'] == 'Spain') & 
    (Marketing_Team_Suggestion['reviewRating'] >= 3)
][['userName', 'Recommended_Attraction']]

Mkt_sug


,userName,Recommended_Attraction
0,Maurice M@MauriceM171,Ponte de Dom Luís I
1,nickchamb@nickchamb,Ponte de Dom Luís I
2,Arthur Villon@arthur4588,Ponte de Dom Luís I
3,James U@696jamesu,Ponte de Dom Luís I
4,Kamal K Sirra@Sirra63,Ponte de Dom Luís I
...,...,...
78300,paulawiddowson@paulawiddowson,Park and National Palace of Pena
78301,Alex L@alxledesma,Park and National Palace of Pena
78302,Hélio R@Quest384404,Park and National Palace of Pena
78303,traveler092006@traveler092006,Park and National Palace of Pena


Since what we did was get a list of all users that have visited Spain and haven't visited Portugal, it means that all the reviews were the country isn't Spain, we will have duplicate user names that we don't need for our purpose.

Also, we are removing all the reviews ratings bellow 3, since that means that the turists were not satisfied with the attraction so it might not be a good idea to recommend it.

In [23]:
Mkt_sug.to_excel('Marketing_Team_Suggestion.xlsx', index=False)